# The primal problem

The primal problem
$$
\begin{array}{lcl}
\mbox{maximize}    & \hat r                                    & \\
\mbox{st}          & \left( \begin{array}{c}
                                0       \\
                                p^i    \\
                            \end{array} \right )
                            - I \left( \begin{array}{c}
                                           \hat r    \\
                                           x         \\ 
                                       \end{array} \right ) \in \mathcal{K}_q,  & \forall i.\\
\end{array}
$$

In [27]:
import random
import sys
import timeit
import mosek

from mosek.fusion import *

random.seed(278) # Makes the script deterministic

d = 2      # Dimmension     
n = 3  # Number of points

# Generate the points
p = [[random.gauss(0.,1.) for dd in range(d)] for nn in range(n)]

def buildprimalandsolve():
    with Model("minimal ball enclosing a set of points - primal formulation") as P:

        hatr = P.variable("hatr", 1, Domain.unbounded())
        x    = P.variable("x",    d, Domain.unbounded())

        # e = Matrix.ones(n,1) # Create a vector of all ones
        e      = [1]*n 
        minuse = [-1]*n

        # Each row of  [-e*hatr,p-e*x'] in a quadratic cone
        P.constraint(Expr.hstack(Expr.outer(minuse,hatr), Expr.sub(p, Expr.outer(e,x))), Domain.inQCone())
        
        P.objective(ObjectiveSense.Maximize,hatr)
        
        P.solve()
        
        print('Radius: %e' % -hatr.level()[0])
        print('Primal objective value: %e Dual objective value: %e' % (P.primalObjValue(),P.dualObjValue()))
        print('x:',)
        print(x.level())

        
buildprimalandsolve()

Radius: 1.074058e+00
Primal objective value: -1.074058e+00 Dual objective value: -1.074058e+00
x:
[ 0.48762009 -0.24860626]


# The dual problem

The dual problem is
$$
\begin{array}{lccl}
\mbox{minimize} &  \sum_i \left [ \begin{array}{c} 0 \\ p^i \end{array} \right ]^T y^i & \\
\mbox{st}       &  \sum_i y^i           & = & \left( \begin{array}{c}
                                                1       \\
                                                0     \\
                                           \end{array} \right ) \\
                & y^i                  & \in \mathcal{K}_q, & \forall i.\\                                   
\end{array}
$$

In [28]:
def buildualandsolve():
    with Model("minimal ball enclosing a set of points - dual formulation") as D:

        # Transposed y
        yt    = D.variable("yt", NDSet(n, 1+d), Domain.inQCone())

        e    = [1.0]*n
        b    = [0.0]*(1+d) 
        b[0] = 1.0    

        c    = D.constraint(Expr.sub(Expr.mul(yt.transpose(),e),b), Domain.equalsTo(0.0))
        
        D.objective(ObjectiveSense.Minimize,Expr.dot(Matrix.dense(p),yt.slice([0, 1],[n, 1+d]))) 
    
        D.solve()
 
        print('Primal objective value: %e Dual objective value: %e' % (D.primalObjValue(),D.dualObjValue()))
        print('Duals on constraints:',)
        print(c.dual())

        
buildualandsolve()
    

Primal objective value: -1.074058e+00 Dual objective value: -1.074058e+00
Duals on constraints:
[-1.07405828  0.48762008 -0.24860626]


Dot product between two matrices is
$$
\begin{array}{rcl}
\mbox{dot}(A,B) & = & <A,B> \\
                & = & A^T B \\
                & = & \sum_i \sum_j A_{ij} B_{ij}.
\end{array}      
$$
Is exploited in the objective.